In [1]:
#Constants, experiment parameters
NUM_QUEENS = 8
POPULATION_SIZE = 10
MIXING_NUMBER = 2
MUTATION_RATE = 0.05

In [2]:
def fitness_score(seq):
    score = 0
    
    for row in range(NUM_QUEENS):
        col = seq[row]
        
        for other_row in range(NUM_QUEENS):
            
            #queens cannot pair with itself
            if other_row == row:
                continue
            if seq[other_row] == col:
                continue
            if other_row + seq[other_row] == row + col:
                continue
            if other_row - seq[other_row] == row - col:
                continue
            #score++ if every pair of queens are non-attacking.
            score += 1
    
    #divide by 2 as pairs of queens are commutative
    return score/2

In [3]:
import random
from scipy import special as sc

def selection(population):
    parents = []
    
    for ind in population:
        #select parents with probability proportional to their fitness score
        if random.randrange(sc.comb(NUM_QUEENS, 2)*2) < fitness_score(ind):
            parents.append(ind)
            
    
    return parents

In [4]:
import itertools

def crossover(parents):
    
    #random indexes to to cross states with
    cross_points = random.sample(range(NUM_QUEENS), MIXING_NUMBER - 1)
    offsprings = []
    
    #all permutations of parents
    permutations = list(itertools.permutations(parents, MIXING_NUMBER))
    
    for perm in permutations:
        offspring = []
        
        #track starting index of sublist
        start_pt = 0
        
        for parent_idx, cross_point in enumerate(cross_points): #doesn't account for last parent
            
            #sublist of parent to be crossed
            parent_part = perm[parent_idx][start_pt:cross_point]
            offspring.append(parent_part)
            
            #update index pointer
            start_pt = cross_point
            
        #last parent
        last_parent = perm[-1]
        parent_part = last_parent[cross_point:]
        offspring.append(parent_part)
        
        #flatten the list since append works kinda differently
        offsprings.append(list(itertools.chain(*offspring)))
    
    return offsprings

In [5]:
def mutate(seq):
    for row in range(len(seq)):
        if random.random() < MUTATION_RATE:
            seq[row] = random.randrange(NUM_QUEENS)
    
    return seq

In [6]:
def print_found_goal(population, to_print=True):
    for ind in population:
        score = fitness_score(ind)
        if to_print:
            print(f'{ind}. Score: {score}')
        if score == sc.comb(NUM_QUEENS, 2):
            if to_print:
                print('Solution found')
            return True
    
    if to_print:
        print('Solution not found')
    return False

In [7]:
def evolution(population):
    #select individuals to become parents
    parents = selection(population)

    #recombination. Create new offsprings
    offsprings = crossover(parents)

    #mutation
    offsprings = list(map(mutate, offsprings))

    #introduce top-scoring individuals from previous generation and keep top fitness individuals
    new_gen = offsprings

    for ind in population:
        new_gen.append(ind)

    new_gen = sorted(new_gen, key=lambda ind: fitness_score(ind), reverse=True)[:POPULATION_SIZE]

    return new_gen

In [8]:
def generate_population():
    population = []

    for individual in range(POPULATION_SIZE):
        new = [random.randrange(NUM_QUEENS) for idx in range(NUM_QUEENS)]
        population.append(new)
    
    return population

In [9]:
#Running the experiment

generation = 0

#generate random population
population = generate_population()
    
while not print_found_goal(population):
    print(f'Generation: {generation}')
    print_found_goal(population)
    population = evolution(population)
    generation += 1

[4, 5, 3, 5, 5, 4, 2, 1]. Score: 18.0
[5, 4, 3, 2, 7, 2, 1, 0]. Score: 17.0
[6, 7, 2, 3, 0, 0, 5, 2]. Score: 21.0
[5, 7, 7, 4, 3, 3, 6, 3]. Score: 21.0
[3, 6, 4, 7, 0, 1, 4, 5]. Score: 23.0
[7, 1, 0, 1, 2, 5, 6, 3]. Score: 19.0
[0, 1, 3, 6, 5, 0, 5, 5]. Score: 20.0
[4, 0, 3, 2, 6, 4, 1, 3]. Score: 21.0
[5, 4, 4, 6, 7, 0, 4, 3]. Score: 18.0
[6, 6, 3, 0, 4, 6, 1, 3]. Score: 22.0
Solution not found
Generation: 0
[4, 5, 3, 5, 5, 4, 2, 1]. Score: 18.0
[5, 4, 3, 2, 7, 2, 1, 0]. Score: 17.0
[6, 7, 2, 3, 0, 0, 5, 2]. Score: 21.0
[5, 7, 7, 4, 3, 3, 6, 3]. Score: 21.0
[3, 6, 4, 7, 0, 1, 4, 5]. Score: 23.0
[7, 1, 0, 1, 2, 5, 6, 3]. Score: 19.0
[0, 1, 3, 6, 5, 0, 5, 5]. Score: 20.0
[4, 0, 3, 2, 6, 4, 1, 3]. Score: 21.0
[5, 4, 4, 6, 7, 0, 4, 3]. Score: 18.0
[6, 6, 3, 0, 4, 6, 1, 3]. Score: 22.0
Solution not found
[3, 6, 4, 7, 0, 1, 4, 5]. Score: 23.0
[4, 0, 5, 2, 6, 4, 1, 3]. Score: 22.0
[6, 6, 3, 0, 4, 6, 1, 3]. Score: 22.0
[7, 7, 0, 1, 2, 5, 6, 0]. Score: 21.0
[7, 1, 0, 1, 2, 6, 6, 3]. Score: 21.

In [11]:
import numpy as np

#checking the mean and stdev of 100,000 random board states
total_sum = []
for i in range(10000):
    population = generate_population()
    for score in list(map(fitness_score, population)):
        total_sum.append(score)
print(f'Mean: {np.mean(total_sum)}')
print(f'St. dev: {np.std(total_sum)}')

Mean: 20.12878
St. dev: 2.379898256564763


In [12]:
gens = []
for run in range(200):
    generation = 0
    population = generate_population()
    # print(f'Run: {run}')
    while not print_found_goal(population, to_print=False):
        population = evolution(population)
        generation += 1
    
    gens.append(generation)
    
print(f'Mean: {np.mean(gens)}')
print(f'St. dev: {np.std(gens)}')

Mean: 2495.94
St. dev: 13885.837044499694


In [13]:
print(f'Min: {min(gens)}')
print(f'Max: {max(gens)}')

Min: 6
Max: 175842


In [14]:
!pip install colorama

In [43]:
from types import resolve_bases
from copy import deepcopy
from colorama import Fore, Back, Style
import numpy as np

# unicode for draw puzzle in command promt or terminal
left_down_angle = '\u2514'
right_down_angle = '\u2518'
right_up_angle = '\u2510'
left_up_angle = '\u250C'

middle_junction = '\u253C'
top_junction = '\u252C'
bottom_junction = '\u2534'
right_junction = '\u2524'
left_junction = '\u251C'

#bar color
bar = Style.BRIGHT + Fore.CYAN + '\u2502' + Fore.RESET + Style.RESET_ALL
dash = '\u2500'

#Line draw code on 8 Puzzle
first_line = Style.BRIGHT + Fore.CYAN + left_up_angle + dash + dash + dash + top_junction + dash + dash + dash + top_junction + dash + dash + dash + top_junction + dash + dash + dash + top_junction + dash + dash + dash + top_junction + dash + dash + dash + top_junction + dash + dash + dash + top_junction + dash + dash + dash + right_up_angle + Fore.RESET + Style.RESET_ALL
middle_line = Style.BRIGHT + Fore.CYAN + left_junction + dash + dash + dash + middle_junction + dash + dash + dash + middle_junction + dash + dash + dash + middle_junction + dash + dash + dash + middle_junction + dash + dash + dash + middle_junction + dash + dash + dash + middle_junction + dash + dash + dash + middle_junction + dash + dash + dash + right_junction + Fore.RESET + Style.RESET_ALL
last_line = Style.BRIGHT + Fore.CYAN + left_down_angle + dash + dash + dash + bottom_junction + dash + dash + dash + bottom_junction + dash + dash + dash + bottom_junction + dash + dash + dash + bottom_junction + dash + dash + dash + bottom_junction + dash + dash + dash + bottom_junction + dash + dash + dash + bottom_junction + dash + dash + dash + right_down_angle + Fore.RESET + Style.RESET_ALL

#puzzle print function
def print_puzzle(array):
    '''
      Function to print puzzle beautifully
    '''
    new_array = []
    for i in array:
        res = [0]*8
        res[i] = 1
        new_array.append(np.array(res))
    
    array = np.array(new_array)
    print(first_line)
    for a in range(len(array)):
        for i in array[a]:
            if i == 1:
                print(bar, Back.RED + ' ' + Back.RESET, end=' ')
            else:
                print(bar, i, end=' ')
        print(bar)
        if a == 7:
            print(last_line)
        else:
            print(middle_line)

In [44]:
print_puzzle([4, 6, 0, 3, 1, 7, 5, 2])

┌───┬───┬───┬───┬───┬───┬───┬───┐
│ 0 │ 0 │ 0 │ 0 │   │ 0 │ 0 │ 0 │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ 0 │ 0 │ 0 │ 0 │ 0 │ 0 │   │ 0 │
├───┼───┼───┼───┼───┼───┼───┼───┤
│   │ 0 │ 0 │ 0 │ 0 │ 0 │ 0 │ 0 │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ 0 │ 0 │ 0 │   │ 0 │ 0 │ 0 │ 0 │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ 0 │   │ 0 │ 0 │ 0 │ 0 │ 0 │ 0 │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ 0 │ 0 │ 0 │ 0 │ 0 │ 0 │ 0 │   │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ 0 │ 0 │ 0 │ 0 │ 0 │   │ 0 │ 0 │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ 0 │ 0 │   │ 0 │ 0 │ 0 │ 0 │ 0 │
└───┴───┴───┴───┴───┴───┴───┴───┘
